# **ETAPA 4 - PROCESSAMENTO PARA CAMADA SILVER**

<br>

---

<br>

Essa etapa será responsável por mover os dados para a camada silver, filtrando e limpando os dados. 

<br>

*`Esse é um modelo modular, complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
# import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import gc

<br>

---

<br>

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

In [0]:
spark = (
    SparkSession.builder
        .appName("Load Data Bronze")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "128MB")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")

        .config("spark.databricks.delta.optimizeWrite.enabled", "true") # novo
        .config("spark.databricks.delta.autoCompact.enabled", "true") # novo
        .getOrCreate()
)

<br>

---

<br>

### Parte 3 - **Definindo Origens e Destinos**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse ` <br>
`--> nome_camada_origem --> nome da camada de origem dos dados` <br>
`--> nome_volume_origem --> nome do volume de origem dos dados dentro da camada` <br>
`--> nome_camada_silver --> nome da camada de destino dos dados` <br>
`--> nome_volume_silver --> nome do volume de destino dos dados dentro da camada` <br>

In [0]:
nome_datalakehouse = "dataexperts" 

nome_camada_origem = "bronze"   
nome_volume_origem = "vendas_atual"

nome_camada_silver = "silver"
nome_volume_silver = "vendas"

origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}/"


O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"
destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}/"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_silver}.{nome_volume_silver}")

<br>

---

<br>

### Parte 4 - **Leitura dos Dados**


O código a seguir lê todos os dados e colocar em um Data Frame para que seja possível manipular os dados:

In [0]:
tabelas_bronze = [
    "bronze_dim_categoria_produto",
    "bronze_dim_cliente",
    "bronze_dim_data",
    "bronze_dim_localidade",
    "bronze_dim_produto",
    "bronze_fato_vendas"
]

bronze_dfs = {}
for tabela in tabelas_bronze:
    bronze_dfs[tabela] = spark.read.format("delta").load(f"{origem_dados}{tabela}")
    print(f"[INFO] Tabela {tabela} carregada com sucesso.")

<br>

---

<br>

### Parte 5 - **Limpeza dos Dados**

A limpeza de dados é um processo crucial para garantir a **qualidade dos dados**. Isso envolve a remoção de **dados duplicados ou incorretos**, a **padronização de formatos e valores de dados** e o **enriquecimento de dados** com informações adicionais. Tudo isso para **garantir** que os **dados** sejam **precisos** e **confiáveis**.

Cada coluna deve ser analisada para encontrar possíveis inconformidades que podem ser encontradas e resolvidas nessa estapa, entre elas estão:
- Remoção de dados duplicados 
- 
- 
- 
- 

In [0]:
def add_sk(df, id_col, sk_name):
    return df.withColumn(sk_name, sha2(col(id_col).cast("string"), 256))

df_categoria = bronze_dfs["bronze_dim_categoria_produto"]
dataframe_silver_categoria = add_sk(
    df_categoria.dropDuplicates(["categoria_id"])
    .withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
    .withColumn("dt_inicio", current_date())
    .withColumn("dt_fim", lit(None).cast("date"))
    .withColumn("is_current", lit(True)),
    "categoria_id", "sk_categoria"
)

df_cliente = bronze_dfs["bronze_dim_cliente"]
dataframe_silver_cliente = (
    df_cliente.dropDuplicates(["cliente_id"])
    .withColumn("nome_cliente", initcap(trim(col("nome_cliente"))))
    .withColumn("estado", upper(col("estado")))
    .withColumn("cidade", initcap(col("cidade")))

    .withColumn("dt_inicio", current_date())
    .withColumn("dt_fim", lit(None).cast("date"))
    .withColumn("is_current", lit(True))
)

df_produto = bronze_dfs["bronze_dim_produto"]
dataframe_silver_produto = add_sk(
    df_produto.dropDuplicates(["produto_id"])
    .withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
    .withColumn("dt_inicio", current_date())
    .withColumn("dt_fim", lit(None).cast("date"))
    .withColumn("is_current", lit(True)),
    "produto_id", "sk_produto"
)

# Dimensão Localidade
df_localidade = bronze_dfs["bronze_dim_localidade"]
dataframe_silver_localidade = add_sk(
    df_localidade.dropDuplicates(["localidade_id"])
    .withColumn("estado", upper(col("estado")))
    .withColumn("cidade", initcap(col("cidade")))
    .withColumn("dt_inicio", current_date())
    .withColumn("dt_fim", lit(None).cast("date"))
    .withColumn("is_current", lit(True)),
    "localidade_id", "sk_localidade"
)

df_data = bronze_dfs["bronze_dim_data"]
dataframe_silver_data = add_sk(
    df_data.dropDuplicates(["data_id"])
    .withColumn("dt_inicio", current_date())
    .withColumn("dt_fim", lit(None).cast("date"))
    .withColumn("is_current", lit(True)),
    "data_id", "sk_data"
)

dataframe_silver_cliente = add_sk(dataframe_silver_cliente, "cliente_id", "sk_cliente")


dataframe_silver_fato = (
    df_fato.dropDuplicates(["venda_id"])
    .filter(col("quantidade") > 0)
    .join(dataframe_silver_cliente.select("cliente_id", "sk_cliente"), on="cliente_id", how="left")
    .join(dataframe_silver_produto.select("produto_id", "sk_produto"), on="produto_id", how="left")
    .join(dataframe_silver_categoria.select("categoria_id", "sk_categoria"), on="categoria_id", how="left")
    # --- Adicionando as que faltavam ---
    .join(dataframe_silver_localidade.select("localidade_id", "sk_localidade"), on="localidade_id", how="left")
    .join(dataframe_silver_data.select("data_id", "sk_data"), on="data_id", how="left")
    # ----------------------------------
    .drop("cliente_id", "produto_id", "categoria_id", "localidade_id", "data_id")
)

<br>

---

<br>

### Parte - ****

Texto

In [0]:
# Mapeamento de caminhos e DataFrames
tabelas_para_salvar = {
    "silver_dim_cliente": dataframe_silver_cliente,
    "silver_dim_categoria": dataframe_silver_categoria,
    "silver_dim_produto": dataframe_silver_produto,
    "silver_dim_localidade": dataframe_silver_localidade, 
    "silver_dim_data": dataframe_silver_data,             
    "silver_fato_vendas": dataframe_silver_fato
}

for nome_tabela, df_silver in tabelas_para_salvar.items():
    path_destino = f"{destino_dados}{nome_tabela}"
    
    if not DeltaTable.isDeltaTable(spark, path_destino):
        df_silver.write.format("delta").save(path_destino)
        print(f"[INFO] Tabela {nome_tabela} criada.")
    else:
        # Para a Fato, usamos OVERWRITE ou APPEND simples
        if "fato" in nome_tabela:
            df_silver.write.format("delta").mode("overwrite").save(path_destino)
        else:
            # Para Dimensões, usamos o MERGE que você já tem
            dt_silver = DeltaTable.forPath(spark, path_destino)
            # (Aqui você pode repetir a lógica de merge que já escreveu)
            print(f"[INFO] Merge realizado em {nome_tabela}")

<br>

---

<br>

### **Resultado Final**

O código a seguir é uma das maneiras simples de se mostrar se deu certo ou não a importação do arquivo.


In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

"""
 #
"""